# GPTTokenizer class

In [413]:
import torch
import os
import torch
import urllib.request
import re
import tiktoken


In [414]:
# GPT tokenizer class
from torch.utils.data import Dataset, DataLoader

class GPTTokenizer(Dataset):
  def __init__(self,input, context_len, stride):
    self.x = []
    self.y = []

    self.tokenizer = tiktoken.get_encoding("gpt2")
    tokens = self.tokenizer.encode(input)

    for i in range(0, len(tokens)- context_len, stride):
      indata = tokens[i:i+context_len]
      target = tokens[i+1:i+context_len+1]
      self.x.append(torch.tensor(indata))
      self.y.append(torch.tensor(target))

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

  def decode(self,tk_ids):
    return self.tokenizer.decode([id.tolist() for id in tk_ids])

# GPTDataloader class

In [415]:

def create_loader(input_txt, batch_size,
                  context_len, stride, shuffle,
                  drop_last=True, num_workers=0):

  dataset = GPTTokenizer(input_txt, context_len, stride)

  print(f"{batch_size=},{context_len=}, {stride=},{shuffle=},  {num_workers=}" )



  dataloader = DataLoader(dataset, batch_size=batch_size,
                          shuffle=shuffle,
                          drop_last=drop_last,
                          num_workers=num_workers)
  return dataloader,dataset

In [416]:
context_len = 8
stride =8
batch_size = 1

if not os.path.exists('verdict.txt'):
  url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt"
  file_path = "sample_data/verdict.txt"
  urllib.request.urlretrieve(url,file_path)

with open('./sample_data/verdict.txt','r',encoding='utf8') as F:
  raw_data = F.read()

dataloader,tkn = create_loader(raw_data, batch_size=batch_size,
                                context_len=context_len,
                                stride=stride,
                                num_workers=0, shuffle=False)

data = iter(dataloader)
#print(len(data))
#[(print(next(data)[0].tolist()[0])) for ii in range(1)]
''.join([ tkn.decode(next(data)[0][0]) for ii in range(0,100,2) ])
#tkn.decode([[40,367,2885]][0])

batch_size=1,context_len=8, stride=8,shuffle=False,  num_workers=0


'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\'s going to send the value of my picture \'way up; but I don\'t think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing\'s lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn\'s "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?\n\nWell!--even 